 <body>
  <h1>Apply trained model to the test set (test_v2.csv)</h1>
  <p style='font-size:15px'>In this note book, we will the trained model to the test set (test_v2.csv).</p>
    <p style='font-size:15px'>All feature engineering parts are identical to the training part.</p>
    <p style='font-size:15px'>In the last step, we predict users' trasactions during the period 2018.12 - 2019.1 and save the prediction as CSV file</p>
    <p style='font-size:15px'>After submitting the prediction, I obtaiend score 0.9100</p>
</body>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import time
from datetime import datetime
from pandas import json_normalize
#from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from zipfile import ZipFile
gc.enable()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/home/username/input/train_v2.csv.zip'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['geoNetwork','device','totals','trafficSource']
    JSON_ARR_COL = ['customDimensions']
    zf = ZipFile(csv_path)
    frames = [tf for tf in pd.read_csv(zf.open('test_v2.csv'),
                                       converters={column: json.loads for column in JSON_COLUMNS},
                                       dtype={'fullVisitorId': 'str'}, # Important!!
                                       chunksize = 50000)]
    
    df = pd.concat(frames,ignore_index=True)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df

In [3]:
X_train = load_df('/Users/hyunkyujun/test_v2.csv.zip')


In [4]:
X_train.shape

(401589, 59)

In [5]:
pd.set_option('display.max_columns', 5000)

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 59 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    401589 non-null  object
 1   customDimensions                                   401589 non-null  object
 2   date                                               401589 non-null  int64 
 3   fullVisitorId                                      401589 non-null  object
 4   hits                                               401589 non-null  object
 5   socialEngagementType                               401589 non-null  object
 6   visitId                                            401589 non-null  int64 
 7   visitNumber                                        401589 non-null  int64 
 8   visitStartTime                                     401589 non-null  int64 
 9   geoN

In [4]:
train = X_train.copy()

In [5]:
transaction_col = ['totals.transactions','totals.transactionRevenue' ,'totals.totalTransactionRevenue']
for c in transaction_col:
    train[c] = train[c].astype(float)
    train[c] = train[c].fillna(0)

In [6]:
import datetime
train['date_'] = train['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]),1))
cnt_srs = train.groupby('date_')['date_'].agg(['count'])
cnt_srs.columns = ["count"]
x = range(cnt_srs.shape[0])


In [7]:
train = train.drop(['hits','customDimensions'],axis=1)
train = train.drop([c for c in train.columns if train[c].nunique(dropna=False)==1], axis=1)
train = train.drop([c for c in train.columns if train[c].nunique(dropna=False)==0], axis=1)

In [8]:
pca_dim_traffic = {'trafficSource.campaign':6,'trafficSource.source': 4, 'trafficSource.medium': 5,
                  'trafficSource.adContent':2,
                  'trafficSource.adwordsClickInfo.page':2, 'trafficSource.adwordsClickInfo.slot':3,
                   'trafficSource.adwordsClickInfo.adNetworkType':3,
                  'trafficSource.adwordsClickInfo.isVideoAd':2, 'trafficSource.isTrueDirect':2
                  }


In [9]:
pca_dim_geoNetwork = {'geoNetwork.continent': 6,'geoNetwork.subContinent': 10, 'geoNetwork.country':10,
                     'geoNetwork.region':10, 'geoNetwork.metro':10
                     } 

In [10]:
pca_dim_device = {'device.browser':8, 'device.operatingSystem':8,
                 'device.isMobile':2, 'device.deviceCategory':3} 

In [11]:
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler



def one_hot_pca(df,column_name,dummy_na,n_comp = None):
    # return Numpy array
    y = pd.get_dummies(df[column_name], prefix=column_name[:5],dummy_na=dummy_na)
    sc = StandardScaler()
    y = pd.DataFrame(sc.fit_transform(y),columns = y.columns,index=y.index)
    if n_comp:
        pca = PCA(n_components = n_comp)
    else:
        pca = PCA(n_components = 'mle')
    y = pd.DataFrame(pca.fit_transform(y), index=y.index,columns=[column_name + str(i) for i in range(n_comp)])
    return y
    
    

In [12]:
def one_hot_pca_batch(df,column_name, dummy_na, batch_size,n_comp = None):
    # return Numpy array
    y = pd.get_dummies(df[column_name], prefix=column_name[:5],dummy_na=dummy_na)
    sc = StandardScaler()
    y = pd.DataFrame(sc.fit_transform(y),columns = y.columns,index=y.index)
    if n_comp:
        n_bachtes = batch_size
        inc_pca = IncrementalPCA(n_components = n_comp)
        for x_batch in np.array_split(y,n_bachtes):
            inc_pca.partial_fit(x_batch)
    else:
        pca = PCA(n_components = 'mle')
    y = pd.DataFrame(inc_pca.transform(y), index=y.index,columns=[column_name + str(i) for i in range(n_comp)])
    return y

In [13]:
def one_hot(df,column_name,dummy_na):
    # return Numpy array
    y = pd.get_dummies(df[column_name], prefix=column_name[:5],dummy_na=dummy_na)
    sc = StandardScaler()
    y = pd.DataFrame(sc.fit_transform(y),columns = y.columns,index=y.index)
    return y
    

In [14]:
def scale(df,column_name):
    # return Numpy array
    #df[column_name] = df[column_name].astype(float)
    sc = StandardScaler()
    df[column_name] = sc.fit_transform(sc.fit_transform(df[[column_name]]))
    

In [15]:
for k,v in pca_dim_device.items():
    y = one_hot_pca(train,k,True,v)
    train = train.drop(k,axis=1)
    train = train.join(y)

In [16]:
for k,v in pca_dim_geoNetwork.items():
    y = one_hot_pca(train,k,True,v)
    train = train.drop(k,axis=1)
    train = train.join(y)

In [17]:
for k,v in pca_dim_traffic.items():
    y = one_hot_pca(train,k,True,v)
    train = train.drop(k,axis=1)
    train = train.join(y)

In [18]:
train = train.drop(['trafficSource.referralPath','trafficSource.keyword',
                    'trafficSource.adwordsClickInfo.gclId',
                   'geoNetwork.city','geoNetwork.networkDomain',
                   'visitStartTime','visitId',
                   'date'],axis=1)

In [19]:
to_be_one_hot = ['channelGrouping','date_']

for c in to_be_one_hot:
    y = one_hot(train,c,train[c].isnull().sum() > 1)
    train = train.drop(c,axis=1)
    train = train.join(y)

In [20]:
train['totals.newVisits'] = train['totals.newVisits'].astype(float)
train['totals.newVisits'].fillna(0, inplace=True)

In [21]:
train['totals.timeOnSite'] = train['totals.timeOnSite'].astype(float)
train['totals.timeOnSite'].fillna(train['totals.timeOnSite'].mean(), inplace=True)

In [22]:
train['totals.bounces'] = train['totals.bounces'].astype(float)
train['totals.bounces'].fillna(0, inplace=True)

In [23]:
train['totals.hits'] = train['totals.hits'].astype(float)
train['totals.pageviews'] = train['totals.pageviews'].astype(float)
train['totals.pageviews'].fillna(0, inplace=True)


In [24]:
train = train.drop(['totals.sessionQualityDim'],axis=1)

In [25]:
to_be_scaled = ['visitNumber','totals.hits','totals.pageviews','totals.timeOnSite',
                'totals.newVisits',
                'totals.transactions',
                'totals.transactionRevenue','totals.totalTransactionRevenue','totals.bounces']
for c in to_be_scaled:
    scale(train,c)

In [26]:
col_order = ['fullVisitorId',
 'visitNumber',
 'totals.hits',
 'totals.pageviews',
 'totals.bounces',
 'totals.newVisits',
 'totals.timeOnSite',
 'totals.transactions',
 'totals.transactionRevenue',
 'totals.totalTransactionRevenue',
 'device.browser0',
 'device.browser1',
 'device.browser2',
 'device.browser3',
 'device.browser4',
 'device.browser5',
 'device.browser6',
 'device.browser7',
 'device.operatingSystem0',
 'device.operatingSystem1',
 'device.operatingSystem2',
 'device.operatingSystem3',
 'device.operatingSystem4',
 'device.operatingSystem5',
 'device.operatingSystem6',
 'device.operatingSystem7',
 'device.isMobile0',
 'device.isMobile1',
 'device.deviceCategory0',
 'device.deviceCategory1',
 'device.deviceCategory2',
 'geoNetwork.continent0',
 'geoNetwork.continent1',
 'geoNetwork.continent2',
 'geoNetwork.continent3',
 'geoNetwork.continent4',
 'geoNetwork.continent5',
 'geoNetwork.subContinent0',
 'geoNetwork.subContinent1',
 'geoNetwork.subContinent2',
 'geoNetwork.subContinent3',
 'geoNetwork.subContinent4',
 'geoNetwork.subContinent5',
 'geoNetwork.subContinent6',
 'geoNetwork.subContinent7',
 'geoNetwork.subContinent8',
 'geoNetwork.subContinent9',
 'geoNetwork.country0',
 'geoNetwork.country1',
 'geoNetwork.country2',
 'geoNetwork.country3',
 'geoNetwork.country4',
 'geoNetwork.country5',
 'geoNetwork.country6',
 'geoNetwork.country7',
 'geoNetwork.country8',
 'geoNetwork.country9',
 'geoNetwork.region0',
 'geoNetwork.region1',
 'geoNetwork.region2',
 'geoNetwork.region3',
 'geoNetwork.region4',
 'geoNetwork.region5',
 'geoNetwork.region6',
 'geoNetwork.region7',
 'geoNetwork.region8',
 'geoNetwork.region9',
 'geoNetwork.metro0',
 'geoNetwork.metro1',
 'geoNetwork.metro2',
 'geoNetwork.metro3',
 'geoNetwork.metro4',
 'geoNetwork.metro5',
 'geoNetwork.metro6',
 'geoNetwork.metro7',
 'geoNetwork.metro8',
 'geoNetwork.metro9',
 'trafficSource.campaign0',
 'trafficSource.campaign1',
 'trafficSource.campaign2',
 'trafficSource.campaign3',
 'trafficSource.campaign4',
 'trafficSource.campaign5',
 'trafficSource.source0',
 'trafficSource.source1',
 'trafficSource.source2',
 'trafficSource.source3',
 'trafficSource.medium0',
 'trafficSource.medium1',
 'trafficSource.medium2',
 'trafficSource.medium3',
 'trafficSource.medium4',
 'trafficSource.adContent0',
 'trafficSource.adContent1',
 'trafficSource.adwordsClickInfo.page0',
 'trafficSource.adwordsClickInfo.page1',
 'trafficSource.adwordsClickInfo.slot0',
 'trafficSource.adwordsClickInfo.slot1',
 'trafficSource.adwordsClickInfo.slot2',
 'trafficSource.adwordsClickInfo.adNetworkType0',
 'trafficSource.adwordsClickInfo.adNetworkType1',
 'trafficSource.adwordsClickInfo.adNetworkType2',
 'trafficSource.adwordsClickInfo.isVideoAd0',
 'trafficSource.adwordsClickInfo.isVideoAd1',
 'trafficSource.isTrueDirect0',
 'trafficSource.isTrueDirect1',
 'chann_(Other)',
 'chann_Affiliates',
 'chann_Direct',
 'chann_Display',
 'chann_Organic Search',
 'chann_Paid Search',
 'chann_Referral',
 'chann_Social',
 'date__2018-05-01',
 'date__2018-06-01',
 'date__2018-07-01',
 'date__2018-08-01',
 'date__2018-09-01',
 'date__2018-10-01']

In [27]:
train = train[col_order]

In [33]:
train.info(verbose=True,max_cols=5000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 120 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   fullVisitorId                                  401589 non-null  object 
 1   visitNumber                                    401589 non-null  float64
 2   totals.hits                                    401589 non-null  float64
 3   totals.pageviews                               401589 non-null  float64
 4   totals.bounces                                 401589 non-null  float64
 5   totals.newVisits                               401589 non-null  float64
 6   totals.timeOnSite                              401589 non-null  float64
 7   totals.transactions                            401589 non-null  float64
 8   totals.transactionRevenue                      401589 non-null  float64
 9   totals.totalTransactionRevenue      

In [57]:
train.head()

,fullVisitorId,visitNumber,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,device.browser0,device.browser1,device.browser2,device.browser3,device.browser4,device.browser5,device.browser6,device.browser7,device.operatingSystem0,device.operatingSystem1,device.operatingSystem2,device.operatingSystem3,device.operatingSystem4,device.operatingSystem5,device.operatingSystem6,device.operatingSystem7,device.isMobile0,device.isMobile1,device.deviceCategory0,device.deviceCategory1,device.deviceCategory2,geoNetwork.continent0,geoNetwork.continent1,geoNetwork.continent2,geoNetwork.continent3,geoNetwork.continent4,geoNetwork.continent5,geoNetwork.subContinent0,geoNetwork.subContinent1,geoNetwork.subContinent2,geoNetwork.subContinent3,geoNetwork.subContinent4,geoNetwork.subContinent5,geoNetwork.subContinent6,geoNetwork.subContinent7,geoNetwork.subContinent8,geoNetwork.subContinent9,geoNetwork.country0,geoNetwork.country1,geoNetwork.country2,geoNetwork.country3,geoNetwork.country4,geoNetwork.country5,geoNetwork.country6,geoNetwork.country7,geoNetwork.country8,geoNetwork.country9,geoNetwork.region0,geoNetwork.region1,geoNetwork.region2,geoNetwork.region3,geoNetwork.region4,geoNetwork.region5,geoNetwork.region6,geoNetwork.region7,geoNetwork.region8,geoNetwork.region9,geoNetwork.metro0,geoNetwork.metro1,geoNetwork.metro2,geoNetwork.metro3,geoNetwork.metro4,geoNetwork.metro5,geoNetwork.metro6,geoNetwork.metro7,geoNetwork.metro8,geoNetwork.metro9,trafficSource.campaign0,trafficSource.campaign1,trafficSource.campaign2,trafficSource.campaign3,trafficSource.campaign4,trafficSource.campaign5,trafficSource.source0,trafficSource.source1,trafficSource.source2,trafficSource.source3,trafficSource.medium0,trafficSource.medium1,trafficSource.medium2,trafficSource.medium3,trafficSource.medium4,trafficSource.adContent0,trafficSource.adContent1,trafficSource.adwordsClickInfo.page0,trafficSource.adwordsClickInfo.page1,trafficSource.adwordsClickInfo.slot0,trafficSource.adwordsClickInfo.slot1,trafficSource.adwordsClickInfo.slot2,trafficSource.adwordsClickInfo.adNetworkType0,trafficSource.adwordsClickInfo.adNetworkType1,trafficSource.adwordsClickInfo.adNetworkType2,trafficSource.adwordsClickInfo.isVideoAd0,trafficSource.adwordsClickInfo.isVideoAd1,trafficSource.isTrueDirect0,trafficSource.isTrueDirect1,chann_(Other),chann_Affiliates,chann_Direct,chann_Display,chann_Organic Search,chann_Paid Search,chann_Referral,chann_Social,date__2018-05-01,date__2018-06-01,date__2018-07-01,date__2018-08-01,date__2018-09-01,date__2018-10-01
0,7460955084541987166,-0.045434,-0.131648,-0.186557,-0.913502,-1.573607,1.970315,-0.123151,-0.023556,-0.02302,-0.754044,-0.030524,-0.008980,-0.001366,0.001467,-0.000970,-0.000020,0.001104,-0.533960,-1.797649,1.549865,-0.496153,-0.151437,0.091174,-0.027227,-0.015620,2.116344,1.755749e-26,2.158321,-0.455662,-1.561020e-13,1.457299,-1.706103,-0.350724,-0.02900,-0.027452,9.273139e-12,-1.239493,2.544027,1.719482,-0.539315,-0.687619,-0.040860,-0.270479,0.370173,-0.138977,-0.065991,1.164551,1.528949,-0.002207,0.205093,0.048204,-0.029396,0.007339,0.042134,-0.130375,-0.082531,1.131292,1.438214,-1.105068,-1.948159,-0.993688,0.833523,0.220790,-1.296173,-0.206546,0.862325,1.610940,1.001668,-0.268663,-0.036328,0.003605,0.009907,-0.009166,0.015906,0.006006,-0.002721,-0.341758,-0.001956,0.000987,0.000323,0.000199,-0.000188,1.201770,0.160623,-0.017190,-0.010585,-1.268781,-0.210509,-0.10757,-0.013870,-0.016937,-0.233431,0.000502,-0.23291,-0.000096,-0.232768,-0.000851,-0.000132,-0.232778,-0.000831,-5.220139e-11,-0.232911,9.316106e-14,1.847139,6.346902e-14,-0.003865,-0.166503,-0.483437,-0.133925,1.012108,-0.181695,-0.417071,-0.318001,1.905582,-0.475864,-0.456947,-0.45389,-0.441647,-0.319575
1,460252456180441002,15.282969,-0.131648,-0.186557,-0.913502,-1.573607,-0.621759,-0.123151,-0.023556,-0.02302,-0.754044,-0.030524,-0.008980,-0.001366,0.001467,-0.000970,-

In [63]:
train_dict = train.groupby('fullVisitorId')[train.columns[1:]].apply(lambda g: g.values.tolist()).to_dict()


In [64]:
import math
from collections import defaultdict
train_dict2 = defaultdict(list)
id_dict = defaultdict(list)
for k,v in train_dict.items():
    temp = np.asarray(v)
    train_dict2[temp.shape[0]].append(temp)
    id_dict[temp.shape[0]].append(k)

In [65]:
for k,v in train_dict2.items():
    train_dict2[k] = np.asarray(v)

In [66]:
for k, v in id_dict.items():
    id_dict[k] = np.asarray(v).reshape(len(v),1)

 <body>
  <h1>Load the saved (and trained) model and make prediction</h1>
</body>

In [67]:
import keras

model = keras.models.load_model('google_revenue_3times_LSTM.h5')

In [68]:
ans_dict = {}
for k,v in train_dict2.items():
    ans_dict[k] = model.predict(v)

In [69]:
for k in id_dict:
    id_dict[k] = np.concatenate((id_dict[k], ans_dict[k]),axis=1)


In [70]:
final_ans = np.concatenate([id_dict[k] for k in id_dict],axis=0)
final_ans[:,0] = final_ans[:,0].astype('str')
final_ans[:,1] = final_ans[:,1].astype(float)

 <body>
  <h1>Make the predictions to CSV format</h1>
  <p style='font-size:15px'>the CSV file has two columns. One is full visitor ID and the other column is the predicted user trasactions</p>
</body>

In [73]:
np.savetxt('google_revenue_5times_range_1to10.csv', final_ans, delimiter=',',
           fmt='%s',header='fullVisitorId,PredictedLogRevenue')

 <body>
  <h1>The score is 0.9100, which is in top 10 percent!</h1>
  <p style='font-size:15px'>The average score for top 30 teams is arond 0.8835</p>
</body>

<body>
      <img src="Screen Shot 2020-09-14 at 1.56.29 PM.png" alt="alternatetext">
   </body>

<body>
      <img src="Screen Shot 2020-09-14 at 2.24.52 PM.png" alt="alternatetext">
   </body>


 <body>
  <h1>You may find the leaderboard page of the competition at </h1>
    <p style='font-size:20px'>https://www.kaggle.com/c/ga-customer-revenue-prediction/leaderboard </p>
</body>